import libraries

In [18]:
import os
import json
import pandas as pd
from utils import load_pickle

Helper Functions

In [6]:
def load_json(path):

    with open(path, "r") as f:

        return json.load(f)

Paths

In [42]:
# Manually add in (hack)
folder = "./results_cache/"
all_folders = []
all_test_performances = {"loss": {}, "jaccard": {}} 

for model in os.listdir(folder):
    subfolder = os.path.join(folder, model)
    for checkpoint in os.listdir(subfolder):
        all_folders.append(os.path.join(subfolder, checkpoint))

for f in all_folders:

    expt_name = f.replace("./results_cache/", "").replace("/", "_")
    test_performance = load_json(os.path.join(f, "test_performance.json"))
    for k, v in test_performance.items():
        all_test_performances[k][expt_name] = round(v, 5)

In [43]:
performance_table = pd.DataFrame.from_dict(all_test_performances)
performance_table

,loss,jaccard
MS_encoder_vanilla,0.76827,0.43814
MS_encoder_cleaned_denoised,0.65370,0.49787
MS_encoder_denoised,0.64973,0.50182
MS_encoder_cleaned,0.77506,0.42994
binned_MS_encoder_cleaned,0.77443,0.42110
binned_MS_encoder_cleaned_denoised,0.64184,0.49857
binned_MS_encoder_denoised,0.63243,0.50567
binned_MS_encoder_vanilla,0.77228,0.42396
formula_encoder_cleaned_denoised,0.64457,0.50089
formula_encoder_denoised,0.63139,0.50815
